In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('datasets/signal/train/train/train001.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
submit = pd.read_csv('datasets/signal/sample_submission.csv')

In [10]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = 'datasets/signal/train/train'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)
big_df

,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,Sleep_Stage
0,-23.52,-30.0,-31.0,49.0,31.15,0.585693,70.67,0.796875,W
1,-26.46,-30.0,-31.0,49.0,31.15,0.585693,70.67,0.796875,W
2,-29.41,-30.0,-32.0,49.0,31.15,0.585693,70.67,0.796875,W
3,-32.24,-30.0,-32.0,49.0,31.15,0.585693,70.67,0.796875,W
4,-34.78,-31.0,-32.0,48.0,31.15,0.585693,70.67,0.796875,W
...,...,...,...,...,...,...,...,...,...
127885435,3.32,46.0,41.0,-14.0,35.50,0.141162,68.07,0.843750,R
127885436,2.59,46.0,41.0,-14.0,35.50,0.141162,68.07,0.843750,R
127885437,1.76,46.0,41.0,-14.0,35.50,0.141162,68.07,0.843750,R
127885438,0.85,45.0,41.0,-14.0,35.50,0.141162,68.07,0.843750,R


In [11]:
from catboost import CatBoostClassifier

In [12]:
big_df.isna().sum()

BVP            0
ACC_X          0
ACC_Y          0
ACC_Z          0
TEMP           0
EDA            0
HR             0
IBI            0
Sleep_Stage    0
dtype: int64

In [13]:
train = big_df.drop('Sleep_Stage',axis=1)
target = big_df['Sleep_Stage']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)

In [115]:
model = CatBoostClassifier(task_type='GPU',learning_rate=0.01)

In [116]:
model.fit(X_train, y_train)

0:	learn: 1.0936938	total: 161ms	remaining: 2m 40s
1:	learn: 1.0914025	total: 317ms	remaining: 2m 38s
2:	learn: 1.0854426	total: 471ms	remaining: 2m 36s
3:	learn: 1.0743046	total: 625ms	remaining: 2m 35s
4:	learn: 1.0686774	total: 776ms	remaining: 2m 34s
5:	learn: 1.0640541	total: 929ms	remaining: 2m 33s
6:	learn: 1.0622247	total: 1.08s	remaining: 2m 33s
7:	learn: 1.0594342	total: 1.23s	remaining: 2m 32s
8:	learn: 1.0546145	total: 1.38s	remaining: 2m 32s
9:	learn: 1.0477035	total: 1.53s	remaining: 2m 31s
10:	learn: 1.0410176	total: 1.68s	remaining: 2m 30s
11:	learn: 1.0361474	total: 1.83s	remaining: 2m 30s
12:	learn: 1.0329431	total: 1.98s	remaining: 2m 30s
13:	learn: 1.0304302	total: 2.13s	remaining: 2m 29s
14:	learn: 1.0272799	total: 2.29s	remaining: 2m 30s
15:	learn: 1.0230602	total: 2.44s	remaining: 2m 30s
16:	learn: 1.0180998	total: 2.58s	remaining: 2m 29s
17:	learn: 1.0130651	total: 2.73s	remaining: 2m 29s
18:	learn: 1.0085946	total: 2.88s	remaining: 2m 28s
19:	learn: 1.0049286	t

In [117]:
from sklearn.metrics import f1_score
f1_score(y_val, model.predict(X_val), average='weighted')

0.6503499835803629

In [98]:
model.get_all_params()

{'nan_mode': 'Min',
 'gpu_ram_part': 0.95,
 'eval_metric': 'MultiClass',
 'iterations': 1000,
 'leaf_estimation_method': 'Newton',
 'observations_to_bootstrap': 'TestOnly',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'devices': '-1',
 'eval_fraction': 0,
 'pinned_memory_bytes': '104857600',
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'gpu_cat_features_storage': 'GpuRam',
 'fold_size_loss_normalization': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'use_best_model': False,
 'meta_l2_frequency': 0,
 'class_names': ['N', 'R', 'W'],
 'random_seed': 0,
 'depth': 6,
 'border_count': 128,
 'min_fold_size': 100,
 'data_partition': 'DocParallel',
 'bagging_temperature': 1,
 'classes_count': 0,
 'auto_class_we

In [19]:
submit.shape

(7970, 2)

In [22]:
submit

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,NaN
4,test001_00004,NaN
...,...,...
7965,test010_00857,NaN
7966,test010_00858,NaN
7967,test010_00859,NaN
7968,test010_00860,NaN


In [120]:
def predict(df):
    p = model.predict(df)
    w = list(p.flatten()).count('W')
    n = list(p.flatten()).count('N')
    r = list(p.flatten()).count('R')
    if w > n:
        if w > r:
          return 'W'
        else:
            return 'R'
    elif n > w:
        if n > r:
          return 'N'
        elif n == r:
            return 'N'
        else : return 'R'
    else :
        return 'N'

In [ ]:
path = 'datasets/signal/test/test_segment/'
result = []
for i in submit['id']:
    idx = i.split('_')[0]
    iid = i.split('_')[1]
    data = pd.read_csv(f'datasets/signal/test/test_segment/{idx}/{i}.csv')
    print(i)
    result.append(predict(data))

test001_00000
test001_00001
test001_00002
test001_00003
test001_00004
test001_00005
test001_00006
test001_00007
test001_00008
test001_00009
test001_00010
test001_00011
test001_00012
test001_00013
test001_00014
test001_00015
test001_00016
test001_00017
test001_00018
test001_00019
test001_00020
test001_00021
test001_00022
test001_00023
test001_00024
test001_00025
test001_00026
test001_00027
test001_00028
test001_00029
test001_00030
test001_00031
test001_00032
test001_00033
test001_00034
test001_00035
test001_00036
test001_00037
test001_00038
test001_00039
test001_00040
test001_00041
test001_00042
test001_00043
test001_00044
test001_00045
test001_00046
test001_00047
test001_00048
test001_00049
test001_00050
test001_00051
test001_00052
test001_00053
test001_00054
test001_00055
test001_00056
test001_00057
test001_00058
test001_00059
test001_00060
test001_00061
test001_00062
test001_00063
test001_00064
test001_00065
test001_00066
test001_00067
test001_00068
test001_00069
test001_00070
test00

In [112]:
result.count('N')

3978

In [109]:
submit['labels'] = result

In [110]:
submit

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,W
4,test001_00004,W
...,...,...
7965,test010_00857,W
7966,test010_00858,W
7967,test010_00859,W
7968,test010_00860,W


In [114]:
submit.to_csv('catboost-R.csv',index=False)